In [1]:
import pandas as pd
import mibian as mb
import yfinance as yf
from datetime import datetime

/Users/jteruya/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Yahoo Finance Ticker Object

In [2]:
stock_ticker = "CCL"
ticker = yf.Ticker(stock_ticker)

## Input DataFrame Helper Function

In [3]:
# Helper Functions for Constant Fields
def _constant_fields(df, expiration_date, interest_rate):
    df['underlying_price'] = ticker.history(period="0d").iloc[0]['Close']
    df['interest_rate'] = interest_rate
    df['days_to_expiration'] = (datetime.strptime(expiration_date, '%Y-%m-%d') - datetime.now()).days + 1

# Generate Mibian Input DF for Call Chains
def call_chain_input(ticker, expiration_date, interest_rate):
    df = pd.DataFrame(ticker.option_chain(expiration_date).calls[['strike', 'lastPrice']])
    _constant_fields(df, expiration_date, interest_rate)
    return df

# Generate Mibian Input DF for Put Chains
def put_chain_input(ticker, expiration_date, interest_rate):
    df = pd.DataFrame(ticker.option_chain(expiration_date).puts[['strike', 'lastPrice']])
    _constant_fields(df, expiration_date, interest_rate)
    return df

## Black Scholes Assumptions

In [4]:
## Assume volatility is 20
vol = 20

## Assume free interest rate is 3%
interest_rate=3

## Create Input DataFrame Across Multiple Expirations

In [5]:
df = pd.DataFrame()
for expiration_date in ticker.options:
    df = df.append(call_chain_input(ticker=ticker, expiration_date=expiration_date, interest_rate=interest_rate), ignore_index=True)

## Generate Black Scholes Objects and Values

In [6]:
def black_scholes_vol(row, vol):
    return mb.BS(row.tolist(), volatility=vol)

df['ticker'] = stock_ticker
df['black_scholes'] = df.apply(lambda x: black_scholes_vol(x, vol), axis=1)
df['black_scholes_call_theta'] = df['black_scholes'].map(lambda x: x.callTheta)
df['black_scholes_call_rho'] = df['black_scholes'].map(lambda x: x.callRho)
df['black_scholes_vega'] = df['black_scholes'].map(lambda x: x.vega)
df['black_scholes_gamma'] = df['black_scholes'].map(lambda x: x.gamma)

## Preview Output

In [7]:
df.head(10)

,strike,lastPrice,underlying_price,interest_rate,days_to_expiration,ticker,black_scholes,black_scholes_call_theta,black_scholes_call_rho,black_scholes_vega,black_scholes_gamma
0,1.0,17.70,17.83,3,5,CCL,<mibian.BS object at 0x12a6ed5b0>,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2.0,15.30,17.83,3,5,CCL,<mibian.BS object at 0x12a6d8220>,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,8.0,8.80,17.83,3,5,CCL,<mibian.BS object at 0x12a6d8a30>,-1.587548e-08,7.812171e-11,4.623353e-09,4.394593e-06
3,9.0,8.40,17.83,3,5,CCL,<mibian.BS object at 0x12a60cee0>,-4.102627e-03,6.893634e-04,1.653212e-06,1.241610e-03
4,9.5,8.40,17.83,3,5,CCL,<mibian.BS object at 0x12a60c640>,-4.097334e-03,6.893999e-04,1.849400e-13,1.246594e-10
5,10.0,8.98,17.83,3,5,CCL,<mibian.BS object at 0x12a60cdf0>,-4.380245e-03,7.370013e-04,4.786851e-11,2.912001e-08
6,10.5,11.90,17.83,3,5,CCL,<mibian.BS object at 0x12a60ceb0>,-1.078861e-12,4.113374e-15,3.163240e-13,1.745401e-10
7,11.0,9.90,17.83,3,5,CCL,<mibian.BS object at 0x110b950d0>,-4.829001e-03,8.125070e-04,1.094290e-10,5.501593e-08
8,11.5,8.50,17.83,3,5,CCL,<mibian.BS object at 0x12a60c670>,-4.146111e-03,6.976071e-04,4.121581e-64,1.895875e-61
9,12.0,7.26,17.83,3,5,CCL,<mibian.BS object at 0x12a60c7c0>,-3.541267e-03,5.958385e-04,5.731472e-171,2.421282e-168
